In [ ]:
# 일반 회원들의 sequence 추출

# 이슈 회원들의 sequence 추출

# 일반 회원과 이슈 회원을 SID(x-axis), Support(y-axis) 별로 비교

In [ ]:
# 글로벌 하게 사용할 객체 선언

# sid corpus
sid_list = {}

# SDB (temporary memory storage)
sdb = {}

In [14]:
# Load the data
import csv
import pandas as pd

# Non issue users
no_issue_seq_data = pd.read_csv('./data/IP_SPLIT_NO_ISSUE_USERS_SEQ.csv')
# Issue users
issue_seq_data = pd.read_csv('./data/IP_SPLIT_ISSUE_USERS_SEQ.csv')

In [3]:
# Data of the non issue users
no_issue_seq_data.head(10)

,ID,IP,SEQUENCE_PARSE
0,1,119.67.115,"<76,77,73,24,21,21,78,161,101,152,100,152,59,1..."
1,2,211.104.55,"<161,24,24,161,24,161,101,152,100,152,161,4,16..."
2,3,1.235.19,"<161,78,80,76,77,73,27,101,150,100,150,106,80,..."
3,4,66.249.69,"<161,4,161,4,161,4,161,4,161,4,161,4,161,4,161..."
4,5,46.229.168,<161>
5,6,211.39.150,"<161,4,161,4,161,4,161,4,161,4,161,4,161,4,161..."
6,7,103.67.235,<161>
7,8,66.249.75,"<161,157,161,4,161,4,161,4,161,4,161,4,161,4,1..."
8,9,115.139.19,"<161,152,78,80,161,78,80>"
9,10,159.203.125,<161>


In [15]:
# Note. 현재 데이터가 유니크하게 존재하지 않는 문제가 있음 
# Data of the issue users
issue_seq_data.head(10)

,ID,USER_ID,IP,SEQUENCE_PARSE
0,1,21353,211.188.192,"<102,101,103,154,115,115,103,115,154,115,102,1..."
1,2,21353,211.188.192,"<102,101,103,154,115,115,103,115,154,115,102,1..."
2,3,21353,211.188.192,"<102,101,103,154,115,115,103,115,154,115,102,1..."
3,4,21353,211.188.192,"<102,101,103,154,115,115,103,115,154,115,102,1..."
4,5,21353,211.188.192,"<102,101,103,154,115,115,103,115,154,115,102,1..."
5,6,21353,211.188.192,"<102,101,103,154,115,115,103,115,154,115,102,1..."
6,7,21353,211.188.192,"<102,101,103,154,115,115,103,115,154,115,102,1..."
7,8,21353,211.188.192,"<102,101,103,154,115,115,103,115,154,115,102,1..."
8,9,21353,211.188.192,"<102,101,103,154,115,115,103,115,154,115,102,1..."
9,10,21353,211.188.192,"<102,101,103,154,115,115,103,115,154,115,102,1..."


In [7]:
# 시퀸스의 시작, 종료 토큰을 제외한 아이템을 콤마(',') 를 기준으로 분리하여
# 아이템 배열을 만들어서 리턴해준다
def sequence_tokenize(seq):
    return seq.replace('<', '').replace('>', '').split(',')

In [8]:
# bi, tri 등 어떤 ngram 을 만들지와 시퀸스 하나를 받아
# ngram dictionary 를 만들어서 리턴해준다
def extract_ngram(n, seq_items):
    result = []
    for idx, item in enumerate(seq_items):
        window_set = seq_items[idx:idx + n]
        if len(window_set) == n:
            result.append(','.join([str(x) for x in window_set]))
    return result

In [9]:
from collections import defaultdict

# 전에 data set 을 받아 ngram 을 만들어 리턴
def extract_ngram_set(n, seq_datas):
    # 아직 나타나지 않은 ngram 을 0 값에서 시작시키기 위해 디폴트 값을 사용
    result = defaultdict(int)
    for index, row in seq_datas.iterrows():
        # ngram 을 추출
        ngram = extract_ngram(n, sequence_tokenize(row['SEQUENCE_PARSE']))
        # ngram dictionary 에 저장
        for item in ngram:
            result[item] += 1
    return pd.DataFrame(result.items(), columns=['sequence', 'support'])

In [ ]:
# 글로벌 sdb 에 추출한 시퀸스들을 등록하는 함수
def save_to_sdb(list):
    

In [11]:
# 비-이슈 회원들의 ngram 을 추출
normal_ngram = extract_ngram_set(5, no_issue_seq_data)
# 가장 많이 나타나는 패턴 순으로 확인하기 위해 support 값 순으로 정렬
normal_ngram.sort_values(['support'], ascending=[False]).head(10)

,sid,support
11284,"161,4,161,4,161",228904
310,"4,161,4,161,4",227343
71932,"150,29,150,29,150",79738
12475,"152,30,152,30,152",50382
97530,"29,150,29,150,29",47638
114599,"30,152,30,152,30",42774
74032,"29,150,150,29,150",30409
60301,"150,29,150,150,29",28027
72632,"29,150,29,150,150",27312
41648,"150,150,29,150,29",26043


In [16]:
# 이슈 회원들의 ngram 을 추출
issue_ngram = extract_ngram_set(5, issue_seq_data)
# 가장 많이 나타나는 패턴 순으로 확인하기 위해 support 값 순으로 정렬
issue_ngram.sort_values(['support'], ascending=[False]).head(10)

,sid,support
608,"103,103,103,103,103",6613
344,"148,148,113,114,148",3652
600,"102,101,102,101,102",3593
19,"101,102,101,102,101",3593
272,"102,101,103,103,103",3112
287,"115,154,115,102,101",2818
2,"102,101,103,115,154",2729
601,"102,101,102,101,103",2681
125,"103,115,154,115,102",2621
140,"101,103,115,154,115",2472


In [24]:
normal_ngram.sort_values(['support'], ascending=[False]).head(10)

,sid,support
11284,"161,4,161,4,161",228904
310,"4,161,4,161,4",227343
71932,"150,29,150,29,150",79738
12475,"152,30,152,30,152",50382
97530,"29,150,29,150,29",47638
114599,"30,152,30,152,30",42774
74032,"29,150,150,29,150",30409
60301,"150,29,150,150,29",28027
72632,"29,150,29,150,150",27312
41648,"150,150,29,150,29",26043


In [25]:
normal_ngram.head(10)

,sid,support
0,"147,150,100,150,29",1
1,"115,147,89,80,83",1
2,"137,13,92,150,137",1
3,"113,147,33,115,148",9
4,"161,27,76,161,27",1
5,"103,147,89,76,86",2
6,"113,147,33,115,147",1
7,"33,115,147,74,147",1
8,"152,130,30,152,4",1
9,"147,89,76,86,13",22


In [23]:
# 분석을 위해 support 를 rel-support 값으로 변환
normal_ngram_support = normal_ngram['support']
normal_ngram_support_sum = normal_ngram['support'].sum()

0          1
1          1
2          1
3          9
4          1
5          2
6          1
7          1
8          1
9         22
10         1
11         6
12         1
13        24
14         4
15         1
16         1
17         1
18         5
19         1
20         3
21         1
22         3
23        22
24         2
25         1
26        11
27         8
28        12
29         1
          ..
116067     1
116068     3
116069    17
116070    36
116071     1
116072     5
116073     4
116074     1
116075     5
116076     1
116077     1
116078     2
116079     1
116080     3
116081     2
116082     1
116083     1
116084     1
116085     2
116086     1
116087     1
116088     1
116089    34
116090     2
116091     1
116092     5
116093     7
116094     1
116095     1
116096     1
Name: support, Length: 116097, dtype: int64